## 1. Импорт зависимостей


In [ ]:
import pandas as pd
import numpy as np
import math
import os
import logging
import joblib
from datetime import datetime
import struct
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

# Настройка устройства
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Используемое устройство: {device}')

## 2. Загрузка и предварительная обработка данных


In [ ]:
def read_binarry(file_path):
    import struct
    import math

    power = []
    age = []
    coordinate_x = []
    coordinate_y = []
    angle_tetta = []
    angle_phi = []
    energy = []
    time = []

    with open(file_path, 'rb') as binary_file:
        # Убедимся, что читаем весь файл, определив количество записей
        # Каждая запись = 1800 * 4 байт = 7200 байт
        binary_file.seek(0, 2) # Переходим в конец файла
        num_bytes = binary_file.tell()
        num_records = num_bytes // (1800 * 4)
        binary_file.seek(0, 0) # Возвращаемся в начало

        for i in range(num_records):
            binary_file.read(4 * 5)

            tetta = struct.unpack('f', binary_file.read(4))[0]
            angle_tetta.append(tetta)

            phi = struct.unpack('f', binary_file.read(4))[0]
            angle_phi.append(phi)

            x0 = struct.unpack('f', binary_file.read(4))[0]
            coordinate_x.append(x0)

            y0 = struct.unpack('f', binary_file.read(4))[0]
            coordinate_y.append(y0)

            binary_file.read(4 * 5)

            power_eas = struct.unpack('f', binary_file.read(4))[0]
            power.append(math.log10(power_eas))

            age_eas = struct.unpack('f', binary_file.read(4))[0]
            age.append(age_eas)

            binary_file.read(4 * 1565)
            energy_release = struct.unpack('f' * 36, binary_file.read(4 * 36))
            energy.append(list(energy_release))

            binary_file.read(4)
            t = struct.unpack('f' * 144, binary_file.read(4 * 144))
            threshold_time = t[::4]
            time.append(list(threshold_time))

    # Собираем всё в DataFrame
    df = pd.DataFrame({
        'power': power,
        'age': age,
        'x': coordinate_x,
        'y': coordinate_y,
        'tetta': angle_tetta,
        'phi': angle_phi,
        'energy': energy,
        'threshold_time': time,
    })

    return df


In [ ]:
file_path = r'/content/drive/MyDrive/spe27p_100k_2022_correct.dat'
df = read_binarry(file_path)
df.head()

In [ ]:
# 1. Нормализация времени
# Для каждого события сдвигаем время так, чтобы первый сигнал приходил в момент t=1.
# Сигналы со временем <= 0 (отсутствующие) получают время 0.
def normalize_time(time_list):
    positive_times = [t for t in time_list if t > 0]
    if not positive_times:
        return [0.0] * len(time_list)

    min_time = min(positive_times)
    return [t - (min_time - 1.0) if t > 0 else 0.0 for t in time_list]

df['threshold_time_norm'] = df['threshold_time'].apply(normalize_time)

# 2. Фильтрация и масштабирование энергии
# Обнуляем энергию для детекторов, где не было сигнала (время = 0).
# Затем масштабируем ненулевые значения энергии для каждого события отдельно.
df['energy_filtered'] = [
    [e if t > 0 else 0.0 for e, t in zip(energy_row, time_row)]
    for energy_row, time_row in zip(df['energy'], df['threshold_time_norm'])
]

def scale_energy(energy_list):
    non_zero_values = np.array([e for e in energy_list if e != 0]).reshape(-1, 1)
    if non_zero_values.shape[0] < 2: # Не можем масштабировать, если меньше 2 значений
        return energy_list

    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(non_zero_values).flatten()

    scaled_row = []
    non_zero_index = 0
    for e in energy_list:
        if e == 0:
            scaled_row.append(0.0)
        else:
            scaled_row.append(scaled_values[non_zero_index])
            non_zero_index += 1
    return scaled_row

df['energy_scaled'] = df['energy_filtered'].apply(scale_energy)

print("Пример обработанных данных для одной строки:")
print("Время:", df.loc[0, 'threshold_time_norm'])
print("Энергия:", df.loc[0, 'energy_scaled'])

## 3. Класс для обучения и оценки моделей

Это ядро нового решения. Класс `EnsembleModelTrainer` инкапсулирует всю логику, связанную с моделями:
- Подготовка загрузчиков данных (`DataLoader`).
- Определение и создание архитектуры RNN.
- Цикл обучения и валидации для RNN.
- Обучение модели случайного леса (опционально).
- Ассемблирование (усреднение) предсказаний.
- Сохранение всех артефактов: весов, моделей, графиков.
- Ведение лог-файла.

In [ ]:
class EventDataset(Dataset):
    def __init__(self, features_df, targets_series):
        self.features = features_df
        self.targets = targets_series

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        energy = self.features.iloc[index]['energy_scaled']
        time = self.features.iloc[index]['threshold_time_norm']
        target = self.targets.iloc[index]
        input_features = np.stack([energy, time], axis=1)
        return torch.tensor(input_features, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)

class RNNModel(nn.Module):
    # input_size - количество признаков на каждом шаге последовательности (у нас 2: энергия и время)
    def __init__(self, input_size, output_size, hidden_size=256):
        super().__init__()
        self.ReLU = nn.ReLU()
        self.dropout = nn.Dropout(0.33)
        self.hidden_size = hidden_size

        # Определяем RNN слои
        # batch_first=True означает, что тензоры имеют размерность (batch, seq_len, features)
        self.rnn1 = nn.RNN(input_size, 128, batch_first=True)
        self.rnn2 = nn.RNN(128, hidden_size, batch_first=True)
        self.rnn3 = nn.RNN(hidden_size, hidden_size, batch_first=True)

        # Определяем LSTM слои для регрессии
        # Измененные Linear слои на LSTM
        self.lstm1 = nn.LSTM(hidden_size, hidden_size, batch_first=True) # Changed input size to hidden_size
        self.BatchNorm1 = nn.BatchNorm1d(hidden_size)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True) # Changed input size to hidden_size
        self.BatchNorm2 = nn.BatchNorm1d(hidden_size)
        self.lstm3 = nn.LSTM(hidden_size, 128, batch_first=True) # Changed input size to hidden_size and output to 128
        self.BatchNorm3 = nn.BatchNorm1d(128)
        self.out = nn.Linear(128, output_size)

    def forward(self, x):
        # Пропускаем через RNN слои
        # Мы используем выход (output) из RNN, а не скрытое состояние (hidden state)
        x, _ = self.rnn1(x)
        x = self.ReLU(x)
        x = self.dropout(x)
        x, _ = self.rnn2(x)
        x = self.ReLU(x)
        x = self.dropout(x)
        x, _ = self.rnn3(x)
        x = self.ReLU(x)
        x = self.dropout(x)

        # Используем выход последнего временного шага из RNN для входа в LSTM
        # x[:, -1, :] имеет размерность (batch_size, hidden_size)
        # LSTM ожидает (batch, seq_len, features), поэтому добавим фиктивную размерность seq_len=1
        x = x[:, -1, :].unsqueeze(1) # Add seq_len dimension

        # Пропускаем через LSTM слои
        x, _ = self.lstm1(x)
        x = self.BatchNorm1(x.squeeze(1)) # Remove seq_len dimension before BatchNorm
        x = self.ReLU(x)
        x = self.dropout(x)
        x, _ = self.lstm2(x.unsqueeze(1)) # Add seq_len dimension before LSTM
        x = self.BatchNorm2(x.squeeze(1)) # Remove seq_len dimension before BatchNorm
        x = self.ReLU(x)
        x = self.dropout(x)
        x, _ = self.lstm3(x.unsqueeze(1)) # Add seq_len dimension before LSTM
        x = self.BatchNorm3(x.squeeze(1)) # Remove seq_len dimension before BatchNorm
        x = self.ReLU(x)
        x = self.dropout(x)

        # Используем выход последнего временного шага из последнего LSTM для предсказания
        # x имеет размерность (batch_size, 1, 128) после последнего LSTM
        x = self.out(x.squeeze(1)) # Remove seq_len dimension before final Linear layer
        return x

# Основной класс-тренер
class EnsembleModelTrainer:
    def __init__(self, df, target_variable, hidden_size=256, use_ensemble=False,
                 epochs=30, batch_size=128, lr=0.001, run_name=None):
        self.df = df
        self.target_variable = target_variable
        self.hidden_size = hidden_size
        self.use_ensemble = use_ensemble
        self.epochs = epochs
        self.batch_size = batch_size
        self.lr = lr

        # Настройка директорий для сохранения результатов
        if run_name:
            self.run_name = run_name
        else:
            self.run_name = f"{self.target_variable}_{'ensemble' if self.use_ensemble else 'rnn'}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        self.base_dir = os.path.join('runs', self.run_name)
        self.weights_dir = os.path.join(self.base_dir, 'weights')
        self.rnn_epochs_dir = os.path.join(self.weights_dir, 'rnn_epochs')
        self.plots_dir = os.path.join(self.base_dir, 'plots')
        os.makedirs(self.rnn_epochs_dir, exist_ok=True)
        os.makedirs(self.plots_dir, exist_ok=True)

        self._setup_logging()
        self.logger.info(f"Run Name: {self.run_name}")
        self.logger.info(f"Target Variable: {self.target_variable}, Hidden Size: {self.hidden_size}, Use Ensemble: {self.use_ensemble}")

    def _setup_logging(self):
        """Настраивает логирование в файл и консоль."""
        self.logger = logging.getLogger(self.run_name)
        self.logger.setLevel(logging.INFO)
        # Предотвращение дублирования логов при повторном запуске в той же сессии
        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        # Логгер для файла
        fh = logging.FileHandler(os.path.join(self.base_dir, 'training_log.txt'))
        fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        self.logger.addHandler(fh)

        # Логгер для консоли
        sh = logging.StreamHandler()
        sh.setFormatter(logging.Formatter('%(message)s'))
        self.logger.addHandler(sh)

    def _prepare_data(self):
        """Разделяет данные на обучающую и тестовую выборки и создает загрузчики."""
        self.logger.info("Preparing data...")
        features = self.df[['energy_scaled', 'threshold_time_norm']]
        targets = self.df[self.target_variable]

        X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)
        X_train.reset_index(drop=True, inplace=True)
        X_test.reset_index(drop=True, inplace=True)
        self.y_train, self.y_test = y_train.reset_index(drop=True), y_test.reset_index(drop=True)

        # Для PyTorch
        train_dataset = EventDataset(X_train, self.y_train)
        test_dataset = EventDataset(X_test, self.y_test)
        self.train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        self.test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
        self.logger.info(f'Train size: {len(train_dataset)}, Test size: {len(test_dataset)}')

        # Для Scikit-learn, если нужно
        if self.use_ensemble:
            self.logger.info("Preparing flattened data for Random Forest...")
            energy_cols = [f'energy_{i}' for i in range(36)]
            time_cols = [f'time_{i}' for i in range(36)]
            X_train_flat = pd.concat([
                pd.DataFrame(X_train['energy_scaled'].tolist(), columns=energy_cols),
                pd.DataFrame(X_train['threshold_time_norm'].tolist(), columns=time_cols)
            ], axis=1)
            X_test_flat = pd.concat([
                pd.DataFrame(X_test['energy_scaled'].tolist(), columns=energy_cols),
                pd.DataFrame(X_test['threshold_time_norm'].tolist(), columns=time_cols)
            ], axis=1)
            self.X_train_rf, self.X_test_rf = X_train_flat, X_test_flat
        self.logger.info("Data preparation complete.")

    def _train_rnn(self):
        """Основной цикл обучения RNN модели."""
        self.logger.info("Starting RNN model training...")
        self.rnn_model = RNNModel(input_size=2, output_size=1, hidden_size=self.hidden_size).to(device)
        loss_fn = nn.MSELoss()
        optimizer = torch.optim.Adam(self.rnn_model.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=False)

        self.train_losses, self.test_losses = [], []
        best_test_loss = float('inf')

        for epoch in range(self.epochs):
            self.rnn_model.train()
            total_train_loss = 0
            progress_bar = tqdm(self.train_loader, desc=f'Epoch {epoch+1}/{self.epochs} [Train]')
            for inputs, targets in progress_bar:
                inputs, targets = inputs.to(device), targets.unsqueeze(1).to(device)
                optimizer.zero_grad()
                preds = self.rnn_model(inputs)
                loss = loss_fn(preds, targets)
                loss.backward()
                optimizer.step()
                total_train_loss += loss.item()
            avg_train_loss = total_train_loss / len(self.train_loader)
            self.train_losses.append(avg_train_loss)

            self.rnn_model.eval()
            total_test_loss = 0
            with torch.no_grad():
                for inputs, targets in self.test_loader:
                    inputs, targets = inputs.to(device), targets.unsqueeze(1).to(device)
                    preds = self.rnn_model(inputs)
                    total_test_loss += loss_fn(preds, targets).item()
            avg_test_loss = total_test_loss / len(self.test_loader)
            self.test_losses.append(avg_test_loss)

            scheduler.step(avg_test_loss)
            self.logger.info(f"Epoch {epoch+1}/{self.epochs} | Train Loss: {avg_train_loss:.4f} | Test Loss: {avg_test_loss:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}")

            # Сохранение весов каждой эпохи
            torch.save(self.rnn_model.state_dict(), os.path.join(self.rnn_epochs_dir, f'epoch_{epoch+1}.pth'))

            # Сохранение лучшей модели
            if avg_test_loss < best_test_loss:
                best_test_loss = avg_test_loss
                torch.save(self.rnn_model.state_dict(), os.path.join(self.weights_dir, 'best_rnn_model.pth'))
                self.logger.info(f"New best model saved with test loss: {best_test_loss:.4f}")

    def _train_rf(self):
        """Обучает модель случайного леса."""
        if not self.use_ensemble:
            return
        self.logger.info("Starting Random Forest model training...")
        self.rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, oob_score=True)
        self.rf_model.fit(self.X_train_rf, self.y_train)
        self.logger.info(f"Random Forest training complete. OOB Score: {self.rf_model.oob_score_:.4f}")

    def _evaluate(self):
        """Оценивает модели на тестовой выборке и вычисляет метрики."""
        self.logger.info("Evaluating models on test data...")
        # Загружаем лучшую RNN модель для оценки
        self.rnn_model.load_state_dict(torch.load(os.path.join(self.weights_dir, 'best_rnn_model.pth')))
        self.rnn_model.eval()

        rnn_preds = []
        with torch.no_grad():
            for inputs, _ in self.test_loader:
                rnn_preds.extend(self.rnn_model(inputs.to(device)).squeeze().cpu().numpy())
        self.rnn_preds = np.array(rnn_preds)
        self.mse_rnn = mean_squared_error(self.y_test, self.rnn_preds)
        self.logger.info(f"Final RNN Model MSE: {self.mse_rnn:.4f}")

        if self.use_ensemble:
            self.rf_preds = self.rf_model.predict(self.X_test_rf)
            self.mse_rf = mean_squared_error(self.y_test, self.rf_preds)
            self.logger.info(f"Final Random Forest MSE: {self.mse_rf:.4f}")

            self.ensemble_preds = (self.rnn_preds + self.rf_preds) / 2
            self.mse_ensemble = mean_squared_error(self.y_test, self.ensemble_preds)
            self.logger.info(f"Final Ensemble (RNN+RF) MSE: {self.mse_ensemble:.4f}")

    def _plot_results(self):
        """Создает и сохраняет графики обучения и результатов."""
        self.logger.info("Generating and saving plots...")
        # График потерь RNN
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_losses, label='Train Loss')
        plt.plot(self.test_losses, label='Test Loss')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.title(f'RNN Training & Test Loss for {self.target_variable}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.plots_dir, 'rnn_loss_curve.png'))
        plt.close()

        # График предсказаний vs реальные значения
        fig, ax = plt.subplots(1, 2 if self.use_ensemble else 1, figsize=(12 if self.use_ensemble else 6, 6))
        if not self.use_ensemble:
            ax = [ax] # Делаем итерируемым для единообразия

        ax[0].scatter(self.y_test, self.rnn_preds, alpha=0.3, label=f'RNN (MSE: {self.mse_rnn:.2f})')
        ax[0].plot([self.y_test.min(), self.y_test.max()], [self.y_test.min(), self.y_test.max()], 'r--', lw=2, label='Ideal')
        ax[0].set_xlabel('Actual Values')
        ax[0].set_ylabel('Predicted Values')
        ax[0].set_title(f'RNN Predictions vs Actual ({self.target_variable})')
        ax[0].legend()
        ax[0].grid(True)

        if self.use_ensemble:
            ax[1].scatter(self.y_test, self.ensemble_preds, alpha=0.3, color='g', label=f'Ensemble (MSE: {self.mse_ensemble:.2f})')
            ax[1].plot([self.y_test.min(), self.y_test.max()], [self.y_test.min(), self.y_test.max()], 'r--', lw=2, label='Ideal')
            ax[1].set_xlabel('Actual Values')
            ax[1].set_ylabel('Predicted Values')
            ax[1].set_title(f'Ensemble Predictions vs Actual ({self.target_variable})')
            ax[1].legend()
            ax[1].grid(True)

        fig.tight_layout()
        fig.savefig(os.path.join(self.plots_dir, 'predictions_vs_actual.png'))
        plt.close(fig)
        self.logger.info("Plots saved.")

    def _save_models(self):
        """Сохраняет финальные обученные модели."""
        if self.use_ensemble:
            path = os.path.join(self.weights_dir, 'random_forest_model.joblib')
            joblib.dump(self.rf_model, path)
            self.logger.info(f"Random Forest model saved to {path}")
        # Лучшая RNN модель уже сохранена во время обучения
        self.logger.info(f"Best RNN model weights saved to {os.path.join(self.weights_dir, 'best_rnn_model.pth')}")

    def run(self):
        """Запускает полный пайплайн обучения и оценки."""
        self._prepare_data()
        self._train_rnn()
        self._train_rf()
        self._evaluate()
        self._plot_results()
        self._save_models()
        self.logger.info("Training and evaluation pipeline finished successfully.")

## 4. Использование класса


In [ ]:
# Пример 1: Обучение ансамбля (RNN + Random Forest) для предсказания 'x'
trainer_x_ensemble = EnsembleModelTrainer(
    df=df,
    target_variable='x',
    hidden_size=256,
    use_ensemble=True,
    epochs=30
)
trainer_x_ensemble.run()

In [ ]:
target_variables = ['power', 'age', 'x', 'y', 'tetta', 'phi']

for target_var in target_variables:
    print(f"\n{'='*50}")
    print(f"Starting training for target variable: {target_var}")
    print(f"{'='*50}")

    trainer = EnsembleModelTrainer(
        df=df,
        target_variable=target_var,
        hidden_size=256,
        use_ensemble=True,  # Или False, если нужен только RNN
        epochs=30,
        batch_size=128,
        lr=0.001,
        run_name=f'{target_var}_ensemble_run' # Уникальное имя для каждого запуска
    )
    trainer.run()

    print(f"\nFinished training for target variable: {target_var}")
    print(f"{'='*50}\n")

